In [2]:
import numpy as np
import sleap_io as sio
import re
import pandas as pd

from brainglobe_registration.elastix.register import (
    run_registration
)
from brainglobe_registration.utils.utils import open_parameter_file
from skimage.color import rgb2gray
from pathlib import Path


In [3]:
video_path = '../21Jan_007.mp4'
video = sio.load_video(video_path)

video_subsample = video[::8]
video_subsample.shape

(787, 1080, 1920, 3)

In [4]:
file_path = (
    Path.home() / "NIU_dev" / "bg-elastix"
    / "brainglobe_registration"
    / "parameters"
    / "brainglobe_registration"
    / "affine.txt"
)

parameter_lists = [("affine", open_parameter_file(file_path))]
parameter_lists[0][1]["FixedInternalImagePixelType"] = ["float"]
parameter_lists[0][1]["MovingInternalImagePixelType"] = ["float"]

In [5]:
from tqdm.notebook import tqdm

output_file = "out_8frame.csv"

with open(output_file, "w") as f:
    f.write("r11,r21,r12,r22,tx,ty\n")

for i in tqdm(range(1, video_subsample.shape[0])):
    fixed_gray = rgb2gray(video_subsample[i-1])
    moving_gray = rgb2gray(video_subsample[i])
    parameters = run_registration(
        fixed_gray,
        moving_gray,
        parameter_lists,
        filter_images=False
    )

    # Regular expression to find the TransformParameters line
    pattern = r'\(TransformParameters ([\d\.\-e ]+)\)'
    input_string = str(parameters)
    # Search for the pattern in the input string
    match = re.search(pattern, input_string)

    if match:
        # Extract the numbers and convert them to floats
        transform_parameters = list(map(float, match.group(1).split()))
        with open(output_file, "a") as f:
            f.write(",".join(map(str, transform_parameters)))
            f.write("\n")
    else:
        print("TransformParameters not found")

  0%|          | 0/786 [00:00<?, ?it/s]

In [6]:
df = pd.read_csv(output_file)
## Add the first frame row
df = pd.concat([pd.DataFrame({
    'r11': [1],
    'r21': [0],
    'r12': [0],
    'r22': [1],
    'tx': [0],
    'ty': [0]
}), df], ignore_index=True)
df.head()

,r11,r21,r12,r22,tx,ty
0,1.00000,0.000000,0.000000,1.00000,0.00000,0.00000
1,1.00196,0.000576,0.000467,1.00208,9.21419,3.28388
2,1.00197,0.001126,0.000343,1.00184,12.28140,3.34411
3,1.00172,0.000427,0.000195,1.00141,12.82440,3.48496
4,1.00125,0.001054,-0.000171,1.00112,10.31470,3.53160


In [7]:
df['tx_sum'] = df['tx'].cumsum()
df['ty_sum'] = df['ty'].cumsum()

In [8]:
df['tx_sum_int'] = df['tx_sum'].round(0).astype(int)
df['ty_sum_int'] = df['ty_sum'].round(0).astype(int)

x_min = df['tx_sum_int'].min()
x_max = df['tx_sum_int'].max()
y_min = df['ty_sum_int'].min()
y_max = df['ty_sum_int'].max()
print(x_min, x_max, y_min, y_max)

0 3817 -273 596


In [9]:
height, width = video_subsample.shape[1:3]

total_height = y_max - y_min + height
total_width = x_max - x_min + width

print(total_height, total_width)

1949 5737


In [10]:
import numpy as np
fused_image = np.zeros((total_height, total_width, 3), dtype=np.uint8)

for i in range(video_subsample.shape[0] - 1, 0, -1):
    x = df['tx_sum_int'][i] - x_min
    y = df['ty_sum_int'][i] - y_min
    fused_image[y:y+height, x:x+width] = video_subsample[i]

In [11]:
import tifffile

tifffile.imwrite("fused_image.tif", fused_image)